In [ ]:
%env DB_PATH=../../../data/db/srs.db
from db import *
from sqlalchemy import insert, select
import json
DATA_PATH = '../../../data/virbs'

# DB

In [ ]:
with Session(engine) as session:
    session.add(Driver(name="Alani"))
    session.add(Driver(name="Audrey"))
    session.add(Driver(name="Cadence"))
    session.add(Driver(name="Daisy"))
    session.add(Driver(name="Mei Xi"))
    session.commit()

In [ ]:
with Session(engine) as session:
    session.add(Buggy(name="Inviscid", abbreviation="inviscid"))
    session.add(Buggy(name="Kingping II", abbreviation="kp"))
    session.add(Buggy(name="Seraph", abbreviation="seraph"))
    session.add(Buggy(name="Mapambazuko", abbreviation="zuke"))
    session.commit()

In [ ]:
file_list = ["zr/medialist.json", "ir/medialist.json"]
media_lists = {}
for file in file_list:
    with open(f'{DATA_PATH}/{file}') as f:
        media_list = json.load(f)['media']
        match file:
            case "zr/medialist.json":
                media_list = media_list[1:]
            case "ir/medialist.json":
                # Remove first 3 media for ir
                media_list = [m for m in media_list if m['duration'] > 100]
        media_list = sorted(media_list, key=lambda x: x['date'])
        media_lists[file] = media_list
        media_list = None
media_lists

In [ ]:
with Session(engine) as session:
    session.add(RollDate(year=2025, month=9, day=21, type=RollType.WEEKEND))
    session.add(RollDate(year=2025, month=11, day=2, type=RollType.WEEKEND))
    session.commit()

In [ ]:
with Session(engine) as session:
    session.add(Sensor(type="virb", name="Zuke R", abbreviation="zr"))
    session.add(Sensor(type="virb", name="Inviscid R", abbreviation="ir"))
    session.commit()

In [ ]:
def add_roll(session: Session, media: dict, 
             roll_date: RollDate, driver: Driver, buggy: Buggy, sensor: Sensor, 
             roll_num: int, use_thm: bool = False):
    dt = datetime.fromtimestamp(media['date'])
    roll = Roll(roll_date_id=roll_date.id, 
                driver_id=driver.id,
                buggy_id=buggy.id,
                roll_number=roll_num,
                start_time=dt)
    session.add(roll)
    session.flush() # populates roll.id
    vid = media['lowResVideoPath'].split('/')[-1].split('.')[0]
    vid = f"%videos%/{sensor.abbreviation}/{vid}.mp4"
    session.add(RollFile(roll_id=roll.id,
                         sensor_id=sensor.id,
                         type='video_preview',
                         uri=vid))
    
    fit = media['fitURL'].split('/')[-1].split('.')[0]
    fit = f"%fit%/{sensor.abbreviation}/{fit}.fit"
    session.add(RollFile(roll_id=roll.id,
                         sensor_id=sensor.id,
                         type='fit',
                         uri=fit))
    if use_thm:
        thm = media['thumbUrl'].split('/')[-1].split('.')[0]
        thm = f"%thumbnails%/{sensor.abbreviation}/{thm}.jpg"
        session.add(RollFile(roll_id=roll.id,
                            sensor_id=sensor.id,
                            type='thumbnail',
                            uri=thm))
        
    return roll

In [ ]:
roll_nums = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
with Session(engine) as session:
    for file in file_list:
        i = 0
        media_list = media_lists[file]
        match file:
            case "zr/medialist.json":
                roll_date = session.scalars(
                    select(RollDate).where(
                        RollDate.year == 2025, 
                        RollDate.month == 9, 
                        RollDate.day == 21
                    ) 
                ).one() 
                
                sensor = session.scalars(
                    select(Sensor)
                        .where(Sensor.abbreviation == "zr") 
                ).one()
                driver = session.scalars(select(Driver)
                                        .where(Driver.name == "Mei Xi")).one()
                buggy = session.scalars(select(Buggy)
                                        .where(Buggy.abbreviation == "inviscid")).one()
                thm = False
            case "ir/medialist.json":
                roll_date = session.scalars(
                    select(RollDate).where(
                        RollDate.year == 2025,
                        RollDate.month == 11, 
                        RollDate.day == 2
                    ) 
                ).one() 
                
                sensor = session.scalars(
                    select(Sensor)
                        .where(Sensor.abbreviation == "ir") 
                ).one()
                driver = session.scalars(select(Driver)
                                        .where(Driver.name == "Cadence")).one()
                buggy = session.scalars(select(Buggy)
                                        .where(Buggy.abbreviation == "seraph")).one()
                thm = True
    
        for media in media_list:
            r = add_roll(session, media, roll_date, driver, buggy, sensor, roll_nums[i % len(roll_nums)], thm)
            print(r)
            i += 1
    # session.rollback()
    session.commit()

In [ ]:
for roll in media_list:
    vid = roll['lowResVideoPath'].split('/')[-1].split('.')[0]
    vid = f"videos/zuke_r/{vid}.GLV"
    # thum = roll['thumbUrl'].split('/')[-1].split('.')[0]
    fit = roll['fitURL'].split('/')[-1].split('.')[0]
    fit = f"backend/notebooks/data/zr/{fit}.fit"
    print(vid, fit)

In [ ]:
with Session(engine) as session:
    d = session.scalars(select(Driver)
                        .where(Driver.name == "Alani")).one() 
d

In [ ]:
# with get_connection() as conn:
#     conn.execute(insert(driver_table).values(name="Alani"))
#     conn.execute(insert(driver_table).values(name="Audrey"))
#     conn.execute(insert(driver_table).values(name="Cadence"))
#     conn.execute(insert(driver_table).values(name="Daisy"))
#     conn.execute(insert(driver_table).values(name="Mei Xi"))
#     conn.commit()

In [ ]:
# with get_connection() as conn:
#     result = conn.execute(driver_table.select())
#     for row in result:
#         print(row)

# Download

In [ ]:
import json
import requests
from tqdm.notebook import tqdm

with open('./data/ir_11_2/medialist.json') as f:
    media_list = json.load(f)['media'][4:]
media_list

In [ ]:
for media in tqdm(media_list):
    url = media['lowResVideoPath']
    # Stream video to file
    response = requests.get(url)
    filename = url.split('/')[-1].replace('GLV', 'mp4')
    with open(f'../../../videos/ir_11_2/{filename}', 'wb') as f:
        f.write(response.content)